# 🔥 Solving the Burgers' Equation using Physics-Informed Neural Networks (PINNs)

We solve the 1D nonlinear Burgers' equation:

\[ \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2} \]

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cpu')

In [ ]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 1)
        )

    def forward(self, x, t):
        xt = torch.cat([x, t], dim=1)
        return self.net(xt)

In [ ]:
def generate_data(N_f):
    x = torch.rand(N_f, 1, device=device, requires_grad=True)
    t = torch.rand(N_f, 1, device=device, requires_grad=True)
    return x, t

In [ ]:
def compute_loss(model, x, t, nu=0.01/np.pi):
    u = model(x, t)
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    f = u_t + u * u_x - nu * u_xx
    return torch.mean(f**2)

In [ ]:
model = PINN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 2000
N_f = 1000
loss_history = []

for epoch in range(epochs):
    optimizer.zero_grad()
    x_f, t_f = generate_data(N_f)
    loss = compute_loss(model, x_f, t_f)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        loss_history.append(loss.item())
        print(f"Epoch {epoch}, Loss: {loss.item()}")

In [ ]:
plt.plot(range(0, epochs, 100), loss_history)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('PINN Training - Burgers Equation')
plt.grid(True)
plt.show()

In [ ]:
x_pred = torch.linspace(0, 1, 100).reshape(-1, 1).to(device)
t_pred = torch.ones_like(x_pred) * 0.5

with torch.no_grad():
    u_pred = model(x_pred, t_pred).cpu().numpy()

plt.plot(x_pred.cpu().numpy(), u_pred)
plt.xlabel('x')
plt.ylabel('u(x, t=0.5)')
plt.title('Prediction at t = 0.5')
plt.grid(True)
plt.show()